# Get Paper Info
These notebook will get the paper info for each speakers in the conference. The attributes are: Number of papers (by year), first paper year, number of citation (by year), country

In [1]:
%run setup.py

### Load the data

In [2]:
# Append all files in Data folder as a datafrom, and add a column for the file name without the extension and folder name
def append_data(folder):
    all_files = glob.glob(folder + "/*.csv")
    df = pd.DataFrame()
    for file in all_files:
        df_temp = pd.read_csv(file, index_col=None)
        df_temp['File name'] = os.path.splitext(os.path.basename(file))[0]
        df = df.append(df_temp, ignore_index=True)
    df['Conference (short)'] = df['File name'].str.split('_').str[0]
    df = df.drop(["Field","File name"], axis=1)
    try:
        df = df.drop(["Key Note Speaker"], axis=1)
    except:
        pass
    return df

df = append_data('Key Note Speakers')
print("Number of rows:", len(df))
print("Number of unique speakers:", df['Full name'].nunique())
df.tail()

Number of rows: 2044
Number of unique speakers: 1797


,Full name,Year,Sex,Conference (short)
2039,Victor Zue,1995,x,WWW
2040,David Goddeau,1995,x,WWW
2041,Christopher Dobbs,1995,x,WWW
2042,Robert W. Lucky,1995,x,WWW
2043,Thomas Reardon,1995,x,WWW


## OpenAleX API

## dblp.org

In [27]:
# #### Find the URL to the author's page on dblp #####
# dblp_URL = "https://dblp.org/search?q=" + "Victor%20Zue"
# page = requests.get(dblp_URL)
# soup = BeautifulSoup(page.content, "html.parser")
# first_author = soup.find(id="completesearch-authors").find_all("ul",class_ = "result-list")[0].find_all("a",href=True)[0]['href']

# ##### Get info from author #####
# author_page = requests.get(first_author)
# soup_of_speaker = BeautifulSoup(author_page.content, "html.parser")

# Title_list = []
Year_list = []

sections = soup_of_speaker.find(id="publ-section").find_all("div", class_="hide-body")
# for s in range(0,len(sections)):
rows_in_sections = sections[0].find_all('ul', class_="publ-list")[0].findAll(True, {"class":['year','entry inproceedings toc','entry article toc']})
year_index = 0
last_row_idx = 0
for row in range(0,len(rows_in_sections)):
    print("--------",row,"----------")
    if rows_in_sections[row].p == None:
        Year_list.append(rows_in_sections[row].text)
        last_row_idx = row 
        
        
        
        
        # print(rows_in_sections[row].text)
        # print('total rows:',len(rows_in_sections))
        # print('row:',row)
        # print('last_row_idx:',last_row_idx)
        # if row == 0:
        #     year_index += 1
        # else:
        #     # year_index += 1
        #     year_index = row - last_row_idx + 1
        # last_row_idx = row    
        
        # print(year_index)
    
    # Year_list.append((rows_in_sections[row].text)*year_index)
    # print(rows_in_sections[row])    
        
        # print(Year_init)
        # Year_list.append(rows_in_sections[row].text)
    # if rows_in_sections[row].p != None:

Year_list 

# Find title
# sections[0].find_all('ul', class_="publ-list")[0].findAll(True, {"class":['year','entry inproceedings toc']})[1].find_all('span',class_="title")[0].text

# for s in sections:
#     a = s.find_all("ul", class_="publ_list")
#     print(a)

-------- 0 ----------
-------- 1 ----------
-------- 2 ----------
-------- 3 ----------
-------- 4 ----------
-------- 5 ----------
-------- 6 ----------
-------- 7 ----------
-------- 8 ----------
-------- 9 ----------
-------- 10 ----------


['2016', '2015', '2012', '2010']

In [200]:
paper_name = []
paper_year = []
papers = soup_of_speaker.find(id="publ-section").find_all("div", class_="hideable")
for t in range(0,len(papers)): 
    years = papers[t].find_all('li', class_="year")
    #### Append year for each paper
    for y in range(0,len(years)):
        year = years[y].get_text()
        n_papers_in_period = len(papers[t].find_all('cite', class_="data tts-content"))
        paper_year.append(year*n_papers_in_period)
    
    #### Append name of each paper
    for i in range(0, n_papers_in_period):
        paper = papers[t].find_all('cite', class_="data tts-content")[i].find_all('span', class_="title")[0].get_text()
        paper_name.append(paper)

# paper_name

[<ul class="publ-list"><li class="year">2016</li><li class="entry inproceedings toc" id="conf/icalt/ZhangLLZ16" itemscope="" itemtype="http://schema.org/ScholarlyArticle"><link href="https://dblp.org/rdf/schema#Publication" itemprop="additionalType"/><div class="box"><img alt="" src="https://dblp.org/img/n.png" title="Conference and Workshop Papers"/></div><div class="nr" id="c123">[c123]</div><nav class="publ"><ul><li class="drop-down"><div class="head"><a href="https://doi.org/10.1109/ICALT.2016.25"><img alt="" class="icon" itemprop="image" src="https://dblp.org/img/paper.dark.hollow.16x16.png"/></a></div><div class="body"><p><b>view</b></p><ul><li class="ee"><a href="https://doi.org/10.1109/ICALT.2016.25" itemprop="url"><img alt="" class="icon" src="https://dblp.org/img/paper.dark.16x16.png"/>electronic edition via DOI</a></li><li class="ee"><a href="http://doi.ieeecomputersociety.org/10.1109/ICALT.2016.25" itemprop="url"><img alt="" class="icon" src="https://dblp.org/img/paper.dark